# `Step 8: Extraction of Cell Type & Gene Marker | ` 
- 1-) `True Labels`
- 2-) `Best Match`
- 3-) `Send Extracted Ranked Genes`

# 0- `Config`


In [ ]:
try: 
    if manager == 1: 
        print("Manager Mode: ON")
except:
    print("Manager Mode: OFF")
    %run s0_config.ipynb 

# 1- `Load` 

## 1-) `Data`

In [ ]:
#check_point_s1    = "checkpoint/session_s1.h5ad"
#check_point_s2    = "checkpoint/session_s2.h5ad"
#check_point_s3    = "checkpoint/session_s3.h5ad"
#check_point_s4    = "checkpoint/session_s4.h5ad"
#check_point_s5    = "checkpoint/session_s5.h5ad"
#check_point_s6    = "checkpoint/session_s6.h5ad"
#check_point_s7    = "checkpoint/session_s7.h5ad"
#check_point_s8    = "checkpoint/session_s8.h5ad"
#check_point_s9    = "checkpoint/session_s9.h5ad"

adata = sc.read(check_point_s7)   

In [ ]:
adata 

## 2-) `Variables`
- None

In [ ]:
import_name = check_point_s7_variables

with open( access_data_path(target_folder = f"process_0/checkpoint", target_file = import_name + ".pkl" ) , 'rb') as file:
    Variables_Dict_s7 = pickle.load(file)   

target_gene_number      = Variables_Dict_s7['target_gene_number']

selected_obs_cluster_key          = Variables_Dict_s7['selected_obs_cluster_key'] 
selected_uns_ranked_key           = Variables_Dict_s7['selected_uns_ranked_key']
selected_cluster_names            = Variables_Dict_s7['selected_cluster_names'] 
selected_sample_cluster_barcode   = Variables_Dict_s7['selected_sample_cluster_barcode'] 

true_label_obs_cluster_key          = Variables_Dict_s7['true_label_obs_cluster_key'] 
true_label_uns_ranked_key           = Variables_Dict_s7['true_label_uns_ranked_key']
true_label_cluster_names            = Variables_Dict_s7['true_label_cluster_names'] 
true_label_sample_cluster_barcode   = Variables_Dict_s7['true_label_sample_cluster_barcode'] 

## 3-) `Functions`

In [ ]:
adata.obs.T

---
# 3- `Bench`

In [ ]:
subject_n_gene_marker = target_gene_number 

In [ ]:
len(adata.var)

## 1-) `True Labels`

In [ ]:
adata.obs['cell_type'].value_counts() 

In [ ]:
temp      = pd.DataFrame(adata.obs['cell_type'].value_counts()) 
temp_name = temp.index.name 
temp.reset_index(inplace = True) 
# find the cell tpyes which has count 1 
discards = temp[temp["count"] == 1] 
discards_list = list(discards[temp_name].values)
discards_list 

In [ ]:
adata_filtered = adata[~adata.obs['cell_type'].isin(discards_list)].copy() 

In [ ]:
adata_filtered.obs['cell_type']

### 2-) `Change the ENSG names with gene markers`

In [ ]:
"""
true_data             = access_data_path("process_0/data", "rna_single_cell_type.tsv") 
ensembl_to_symbol     = pd.read_csv( true_data, sep='\t' ) 

unique_ensembl_to_symbol = ensembl_to_symbol[['Gene', 'Gene name']].drop_duplicates('Gene')
unique_ensembl_to_symbol['Gene'] = unique_ensembl_to_symbol['Gene'].str.strip()
unique_ensembl_to_symbol['Gene name'] = unique_ensembl_to_symbol['Gene name'].str.strip()
unique_ensembl_to_symbol.set_index('Gene', inplace=True) 
adata.var.index = adata.var.index.map(unique_ensembl_to_symbol['Gene name'].get)
adata.var
"""

In [ ]:
"""
unique_ensembl_to_symbol_DF = unique_ensembl_to_symbol.reset_index() 
unique_ensembl_to_symbol_DF.columns = ["ensembl_id", "gene_symbol"]
"""

## 1-) `True Labels`

In [ ]:
"""
# Create a dictionary mapping ENSG IDs to gene symbols
ensembl_to_symbol_dict = pd.Series(unique_ensembl_to_symbol_DF.gene_symbol.values, index=unique_ensembl_to_symbol_DF.ensembl_id).to_dict()

# Now, when you collect top markers for each cell type, you can use this dictionary to convert the ENSG IDs to gene symbols
top_markers_true_labels = {} 
for cell_type in adata.obs['cell_type'].unique(): 
    # This fetches ENSG identifiers 
    markers = adata.uns[true_label_uns_ranked_key]['names'][cell_type][:subject_n_gene_marker]
    # Convert each ENSG ID to its corresponding gene symbol using the mapping dictionary
    gene_symbols = [ensembl_to_symbol_dict.get(ensg_id, "Unknown") for ensg_id in markers]
    top_markers_true_labels[cell_type] = gene_symbols
"""

In [ ]:
top_markers_true_labels = {} 
for cell_type in adata.obs['cell_type'].unique(): 
    markers = adata.uns[true_label_uns_ranked_key]['names'][cell_type][:subject_n_gene_marker]
    top_markers_true_labels[cell_type] = markers 

In [ ]:
pd.DataFrame(top_markers_true_labels) 

## 2-) `Best Match`

In [ ]:
try:
    adata.obs['majority_voting_best_match'].value_counts() 
except: 
    pass 

In [ ]:
try:
    temp      = pd.DataFrame(adata.obs['majority_voting_best_match'].value_counts()) 
    temp_name = temp.index.name 
    temp.reset_index(inplace = True) 
    # find the cell tpyes which has count 1 
    discards = temp[temp["count"] == 1] 
    discards_list = list(discards[temp_name].values)
    discards_list 
except: 
    pass  

In [ ]:
try:
    adata_filtered = adata[~adata.obs['majority_voting_best_match'].isin(discards_list)].copy() 
except:
    pass 

In [ ]:
try:
    adata_filtered.obs['majority_voting_best_match']
except:
    pass 

In [ ]:
try: 
    current_ranked_key = "ranked_majority_voting_best_match" 


    %memit sc.tl.rank_genes_groups(adata_filtered, groupby = "majority_voting_best_match", method = "wilcoxon", key_added = current_ranked_key, reference = 'rest', n_genes = 100)  # Adjust n_genes as needed 

    # Collect top markers for each cell type
    top_markers_majority_vote = {}
    for cell_type in adata_filtered.obs['majority_voting_best_match'].unique():
        markers = adata_filtered.uns[current_ranked_key]['names'][cell_type][:subject_n_gene_marker]  # Adjust the number as needed
        top_markers_majority_vote[cell_type] = markers
except:
    pass 


In [ ]:
try:
    pd.DataFrame(top_markers_majority_vote)
except:
    pass 

## 3-) `Fix Strings`


In [ ]:
adata.obs["cell_type"]

In [ ]:
"""adata.obs["cell_type"] = adata.obs["cell_type"].str.replace("cell", "", regex=False).str.strip()
adata.obs["cell_type"] = adata.obs["cell_type"].str.replace("cells", "", regex=False).str.strip()
adata.obs["cell_type"]"""

## 4-) `Send Extracted Ranked Genes`


In [ ]:
data_1_name = "top_markers_true_labels"
data_1_target_path = access_data_path(target_folder = f"process_0/data", target_file = data_1_name + ".csv" ) 
data_1_target_path 

try:
    data_2_name = "top_markers_majority_vote"
    data_2_target_path = access_data_path(target_folder = f"process_0/data", target_file = data_2_name + ".csv" ) 
    data_2_target_path  
except:
    data_2_target_path =  data_1_target_path

In [ ]:
data_1_target_path

In [ ]:
data_2_target_path

In [ ]:
pd.DataFrame(top_markers_true_labels).to_csv(data_1_target_path) 
try:
    pd.DataFrame(top_markers_majority_vote).to_csv(data_2_target_path)  
except:
    top_markers_majority_vote = top_markers_true_labels 
    pd.DataFrame(top_markers_majority_vote).to_csv(data_2_target_path)  

---

# `Step 8: Checkpoint` 

#### Operation: Checkpoint (Save) 
Save the result.

In [ ]:
adata.var

In [ ]:
columns_to_convert = ['feature_name', 'feature_reference', 'feature_biotype']
for col in columns_to_convert:
    if col in adata.var.columns:
        adata.var[col] = adata.var[col].astype(str)

adata.var.index = adata.var.index.astype(str)



In [ ]:
adata.write(check_point_s8)

In [ ]:
adata

In [ ]:
Variables_Dict_s8 ={"target_gene_number"      : target_gene_number, 
                    
                    "selected_obs_cluster_key"           : selected_obs_cluster_key,
                    "selected_uns_ranked_key"            : selected_uns_ranked_key,
                    "selected_cluster_names"             : list(pd.unique(adata.obs[selected_obs_cluster_key])), 
                    "selected_sample_cluster_barcode"    : list(pd.unique(adata.obs[selected_obs_cluster_key]))[0], 
                    

                    "true_label_obs_cluster_key"           : true_label_obs_cluster_key,
                    "true_label_uns_ranked_key"            : true_label_uns_ranked_key,
                    "true_label_cluster_names"             : list(pd.unique(adata.obs[true_label_obs_cluster_key])), 
                    "true_label_sample_cluster_barcode"    : list(pd.unique(adata.obs[true_label_obs_cluster_key]))[0], 
                     }

export_name = check_point_s8_variables

with open( access_data_path(target_folder = f"process_0/checkpoint", target_file = export_name + ".pkl" )  , 'wb') as file:
    pickle.dump(Variables_Dict_s8, file)
